<a href="https://colab.research.google.com/github/olaioo/polar-sample-database/blob/master/solution/HackatonIESB2021_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Hackaton IESB 2021.2

## recuperando os dados do repositório


In [99]:
!git clone https://github.com/mendelson/polar-sample-database.git

fatal: destination path 'polar-sample-database' already exists and is not an empty directory.


In [100]:
!ls

polar-sample-database  sample_data


## Imports necessários


In [101]:
from sklearn import linear_model
from sklearn.model_selection import train_test_split
import json
import numpy as np
from datetime import datetime
import pandas as pd
import plotly.express as px

## Funções auxiliares


In [102]:
def readFile(url):
  with open(url, 'r') as myfile:
      data=myfile.read()
  return data

def formatExcludingUTC(str):
  return str[:str.rindex('-')]

def getDateFromStr(str, pattern):
  return datetime.strptime(str, pattern).replace(hour=0, minute=0, second=0, microsecond=0)

def collect(array, field, start, end):
  l = []
  for e in array:
    if (getDateFromStr(formatExcludingUTC(e['datetimeStr']), '%Y-%m-%dT%H:%M:%S') >= getDateFromStr(start, '%Y-%m-%dT%H:%M:%S.%f') and getDateFromStr(formatExcludingUTC(e['datetimeStr']), '%Y-%m-%dT%H:%M:%S') <= getDateFromStr(end, '%Y-%m-%dT%H:%M:%S.%f')) :
      l.append(e[field] if e[field] else 0)
  return l

## Histograma sobre os dados

In [103]:
#dados gerais

filesList = !ls polar-sample-database/dataset/

stats = []

filtered = []

for f in filesList:
  dataExercises = json.loads(readFile('polar-sample-database/dataset/'+f))
  filter = True
  stats.append('TOTAL')
  if 'physicalInformationSnapshot' in dataExercises:
    stats.append('PHYSICAL_INFO')
  else:
    filter = False

  if 'kiloCalories' in dataExercises:
    stats.append('CALORIES_INFO')
  else:
    filter = False


  weatherFiles = !ls polar-sample-database/weather\ dataset | grep {dataExercises['startTime'][:dataExercises['startTime'].index('T')]} | grep .json
  if len(weatherFiles) > 0:
    stats.append("WEATHER_INFO")
  else:
    filter = False


  if filter:
    stats.append("FILTERED")
    filtered.append(dataExercises['exercises'][0]['sport'])


In [104]:
df = pd.DataFrame(stats)

fig = px.histogram(df, x=0, title='Visão geral dos dados', labels={'Visão geral dos dados'})
fig.show()

In [105]:
df = pd.DataFrame(filtered)

fig = px.histogram(df, x=0, title='Visão da barra de filtrados')
fig.show()

## Tentando prever o tipo de esporte com base em informações climáticas

In [106]:
filesList = !ls polar-sample-database/dataset/

print(len(filesList))

records = []

for f in filesList:
  dataExercises = json.loads(readFile('polar-sample-database/dataset/'+f))
  for ex in dataExercises['exercises']:
    weatherFiles = !ls polar-sample-database/weather\ dataset | grep {ex['startTime'][:ex['startTime'].index('T')]} | grep .json
    if len(weatherFiles) > 0:
      dataWeather = json.loads(readFile('polar-sample-database/weather dataset/'+weatherFiles[0]))
      values = list(dataWeather['locations'].values())[0]['values']
      records.append((ex['sport'], ex['startTime'], ex['stopTime'], np.nanmean(collect(values, 'temp', ex['startTime'], ex['stopTime'])), np.mean(collect(values, 'wspd', ex['startTime'], ex['stopTime'])), np.mean(collect(values, 'humidity', ex['startTime'], ex['stopTime'])), np.mean(collect(values, 'cloudcover', ex['startTime'], ex['stopTime']))))


print(records)

print(len(records))

152
[('RUNNING', '2016-01-01T18:52:47.797', '2016-01-01T19:26:47.565', 31.16666666666667, 12.338888888888889, 65.40444444444444, 57.02222222222221), ('RUNNING', '2016-01-09T16:58:39.000', '2016-01-09T17:47:59.250', 22.200000000000006, 11.86, 82.36449999999999, 91.5), ('RUNNING', '2016-01-31T09:22:51.000', '2016-01-31T10:32:48.875', 22.868181818181824, 12.509090909090908, 63.876363636363635, 34.09090909090909), ('RUNNING', '2016-02-14T15:55:42.000', '2016-02-14T16:53:43.500', 24.43333333333334, 11.152380952380952, 72.8847619047619, 75.71428571428571), ('RUNNING', '2016-02-21T15:33:10.000', '2016-02-21T16:46:49.375', 23.62380952380953, 11.223809523809525, 70.2147619047619, 48.095238095238095), ('RUNNING', '2016-04-30T10:30:38.000', '2016-04-30T11:05:03.875', 21.10000000000001, 7.617391304347826, 80.79695652173913, 32.608695652173914), ('RUNNING', '2016-05-08T08:05:49.000', '2016-05-08T09:24:22.375', 21.909523809523815, 9.380952380952381, 57.188095238095244, 7.142857142857143), ('RUNNING'

In [107]:
#Creating the X and Y variables
X = []
y = []
for record in records:
  y.append(record[0])
  X.append(list(record[3:]))

#Splitting the recors into training and test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

#Regression with scktlearn
clf = linear_model.LogisticRegression(max_iter=10000).fit(X_train, y_train)
clf.score(X_test, y_test)


0.6666666666666666

##Tentando prever o gasto calórico com base em (duração, tipo de esporte, informções do clima)


In [108]:
filesList = !ls polar-sample-database/dataset/

print(len(filesList))

records = []

token = {}

for f in filesList:
  dataExercises = json.loads(readFile('polar-sample-database/dataset/'+f))
  if('kiloCalories' in dataExercises):
    for ex in dataExercises['exercises']:
      weatherFiles = !ls polar-sample-database/weather\ dataset | grep {ex['startTime'][:ex['startTime'].index('T')]} | grep .json
      if len(weatherFiles) > 0:
        dataWeather = json.loads(readFile('polar-sample-database/weather dataset/'+weatherFiles[0]))
        values = list(dataWeather['locations'].values())[0]['values']
        token[ex['sport']] = token[ex['sport']] if ex['sport'] in token else len(token)
        records.append((dataExercises['kiloCalories'], token[ex['sport']], (datetime.strptime(ex['stopTime'], '%Y-%m-%dT%H:%M:%S.%f') - datetime.strptime(ex['startTime'], '%Y-%m-%dT%H:%M:%S.%f')).total_seconds() , np.nanmean(collect(values, 'temp', ex['startTime'], ex['stopTime'])), np.mean(collect(values, 'wspd', ex['startTime'], ex['stopTime'])), np.mean(collect(values, 'humidity', ex['startTime'], ex['stopTime'])), np.mean(collect(values, 'cloudcover', ex['startTime'], ex['stopTime']))))


print(records)

print(len(records))

print(token)

152
[(445, 0, 2960.25, 22.200000000000006, 11.86, 82.36449999999999, 91.5), (607, 0, 4197.875, 22.868181818181824, 12.509090909090908, 63.876363636363635, 34.09090909090909), (763, 0, 3481.5, 24.43333333333334, 11.152380952380952, 72.8847619047619, 75.71428571428571), (834, 0, 4419.375, 23.62380952380953, 11.223809523809525, 70.2147619047619, 48.095238095238095), (516, 0, 2065.875, 21.10000000000001, 7.617391304347826, 80.79695652173913, 32.608695652173914), (1066, 0, 4713.375, 21.909523809523815, 9.380952380952381, 57.188095238095244, 7.142857142857143), (394, 0, 1957.75, 20.415789473684214, 18.331578947368417, 51.00315789473684, 12.631578947368421), (419, 0, 2411.5, 21.616666666666667, 5.9444444444444455, 43.03166666666666, 0.0), (248, 1, 1945.875, 20.383333333333336, 15.174999999999999, 53.13833333333333, 14.829166666666666), (84, 2, 1992.25, 22.605882352941176, 4.96470588235294, 55.41, 14.905882352941175), (465, 0, 2885.789, 22.605882352941176, 4.96470588235294, 55.41, 14.905882352

In [109]:
#Creating the X and Y variables
X = []
y = []
for record in records:
  y.append(record[0])
  X.append(list(record[1:]))

#Splitting the recors into training and test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

#Regression with scktlearn
clf = linear_model.LinearRegression(normalize=True).fit(X_train, y_train)
print(clf.score(X_test, y_test))
print(clf.predict([[4, 1480.125, 30.200000000000003, 3.6, 76.24, 55.0]]))

0.7145503163556874
[87.29107809]


##Testando se os atributos do clima influenciam ou não, i.e. removendo eles da entrada


In [110]:
filesList = !ls polar-sample-database/dataset/

print(len(filesList))

records = []

token = {}

for f in filesList:
  dataExercises = json.loads(readFile('polar-sample-database/dataset/'+f))
  if('kiloCalories' in dataExercises):
    for ex in dataExercises['exercises']:
      weatherFiles = !ls polar-sample-database/weather\ dataset | grep {ex['startTime'][:ex['startTime'].index('T')]} | grep .json
      if len(weatherFiles) > 0:
        dataWeather = json.loads(readFile('polar-sample-database/weather dataset/'+weatherFiles[0]))
        values = list(dataWeather['locations'].values())[0]['values']
        token[ex['sport']] = token[ex['sport']] if ex['sport'] in token else len(token)
        records.append((dataExercises['kiloCalories'], token[ex['sport']], (datetime.strptime(ex['stopTime'], '%Y-%m-%dT%H:%M:%S.%f') - datetime.strptime(ex['startTime'], '%Y-%m-%dT%H:%M:%S.%f')).total_seconds()))


print(records)

print(len(records))

print(token)

152
[(445, 0, 2960.25), (607, 0, 4197.875), (763, 0, 3481.5), (834, 0, 4419.375), (516, 0, 2065.875), (1066, 0, 4713.375), (394, 0, 1957.75), (419, 0, 2411.5), (248, 1, 1945.875), (84, 2, 1992.25), (465, 0, 2885.789), (208, 3, 2581.625), (191, 4, 2751.394), (607, 4, 7510.192), (77, 4, 1480.125), (1120, 5, 5898.227), (211, 5, 2566.504), (493, 5, 2830.0), (716, 5, 4178.458), (1260, 5, 6487.846), (689, 5, 3547.517), (1557, 5, 7674.962), (301, 0, 1914.816), (920, 5, 4904.748), (1027, 5, 5048.426), (766, 5, 7104.093), (108, 2, 2849.375), (566, 5, 6777.222), (89, 2, 2409.5), (384, 5, 5154.722), (223, 0, 2314.576), (70, 5, 885.001), (450, 5, 2595.107), (659, 5, 3546.171), (413, 0, 1833.875), (921, 5, 4775.693)]
36
{'RUNNING': 0, 'OPEN_WATER_SWIMMING': 1, 'STRENGTH_TRAINING': 2, 'OTHER_OUTDOOR': 3, 'WALKING': 4, 'CYCLING': 5}


In [111]:
#Creating the X and Y variables
X = []
y = []
for record in records:
  y.append(record[0])
  X.append(list(record[1:]))

#Splitting the recors into training and test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

#Regression with scktlearn
clf = linear_model.LinearRegression(normalize=True).fit(X_train, y_train)
print(clf.score(X_test, y_test))
print(clf.predict([[0, 2960.25]]))

0.5950733127996843
[486.4604288]


## Invertendo, se é possível determinar a duração pelo clima e a quantidade de calorias alvo!

In [112]:
filesList = !ls polar-sample-database/dataset/

print(len(filesList))

records = []

token = {}

for f in filesList:
  dataExercises = json.loads(readFile('polar-sample-database/dataset/'+f))
  if('kiloCalories' in dataExercises):
    for ex in dataExercises['exercises']:
      weatherFiles = !ls polar-sample-database/weather\ dataset | grep {ex['startTime'][:ex['startTime'].index('T')]} | grep .json
      if len(weatherFiles) > 0:
        dataWeather = json.loads(readFile('polar-sample-database/weather dataset/'+weatherFiles[0]))
        values = list(dataWeather['locations'].values())[0]['values']
        token[ex['sport']] = token[ex['sport']] if ex['sport'] in token else len(token)
        records.append(((datetime.strptime(ex['stopTime'], '%Y-%m-%dT%H:%M:%S.%f') - datetime.strptime(ex['startTime'], '%Y-%m-%dT%H:%M:%S.%f')).total_seconds(), dataExercises['kiloCalories'], token[ex['sport']], np.nanmean(collect(values, 'temp', ex['startTime'], ex['stopTime'])), np.mean(collect(values, 'wspd', ex['startTime'], ex['stopTime'])), np.mean(collect(values, 'humidity', ex['startTime'], ex['stopTime'])), np.mean(collect(values, 'cloudcover', ex['startTime'], ex['stopTime']))))


print(records)

print(len(records))

print(token)

152
[(2960.25, 445, 0, 22.200000000000006, 11.86, 82.36449999999999, 91.5), (4197.875, 607, 0, 22.868181818181824, 12.509090909090908, 63.876363636363635, 34.09090909090909), (3481.5, 763, 0, 24.43333333333334, 11.152380952380952, 72.8847619047619, 75.71428571428571), (4419.375, 834, 0, 23.62380952380953, 11.223809523809525, 70.2147619047619, 48.095238095238095), (2065.875, 516, 0, 21.10000000000001, 7.617391304347826, 80.79695652173913, 32.608695652173914), (4713.375, 1066, 0, 21.909523809523815, 9.380952380952381, 57.188095238095244, 7.142857142857143), (1957.75, 394, 0, 20.415789473684214, 18.331578947368417, 51.00315789473684, 12.631578947368421), (2411.5, 419, 0, 21.616666666666667, 5.9444444444444455, 43.03166666666666, 0.0), (1945.875, 248, 1, 20.383333333333336, 15.174999999999999, 53.13833333333333, 14.829166666666666), (1992.25, 84, 2, 22.605882352941176, 4.96470588235294, 55.41, 14.905882352941175), (2885.789, 465, 0, 22.605882352941176, 4.96470588235294, 55.41, 14.905882352

In [113]:
#Creating the X and Y variables
X = []
y = []
for record in records:
  y.append(record[0])
  X.append(list(record[1:]))

#Splitting the recors into training and test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

#Regression with scktlearn
clf = linear_model.LinearRegression(normalize=True).fit(X_train, y_train)
print(clf.score(X_test, y_test))
print(clf.predict([[301, 0, 21.452, 6.68, 46.407999999999994, 13.54]]))

0.8862809240780773
[1621.84528616]


## Testando com "data augmentation"

In [114]:
#Creating the X and Y variables
X = []
y = []
for record in records:
  y.append(record[0])
  X.append(list(record[1:]))

X.extend(X)
X.extend(X)
X.extend(X)
X.extend(X)
y.extend(y)
y.extend(y)
y.extend(y)
y.extend(y)

print(len(X))

#Splitting the recors into training and test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

#Regression with scktlearn
clf = linear_model.LinearRegression(normalize=True).fit(X_train, y_train)
print(clf.score(X_test, y_test))
print(clf.predict([[301, 0, 21.452, 6.68, 46.407999999999994, 13.54]]))

576
0.7564886473960362
[1662.92613282]
